In [1]:
import pandas as pd
import plotly.express as px
import glob
from src.walking_recording import WalkingRecording

In [2]:
def get_metrics_df():
    files = glob.glob("data/processed/*/*.csv")
    recordings = [WalkingRecording.from_csv(f) for f in files]
    metrics = []
    for recording in recordings:
        metrics.append(
            {
                "Nom": recording.name,
                "Fichier source": recording.file_name,
                "Longueur de l'enregistrement (s)": recording.get_recording_length(),
                "Nombre de pas": len(recording.get_steps()),
                "Fréquence des pas par fft (Hz)": recording.get_frequency_from_fft(),
                "Fréquence des pas par comptage (Hz)": recording.get_frequency_from_counting_steps(),
                "Amplitude de l'accélération (g)": recording.get_acceleration_amplitude(),
                "Ecart-type de la durée des pas (s)" : recording.get_step_duration_std(),
            }
        )
    return pd.DataFrame(metrics)

df = get_metrics_df()
df.to_csv("data/metrics.csv", index=False)
df.head()

,Nom,Fichier source,Longueur de l'enregistrement (s),Nombre de pas,Fréquence des pas par fft (Hz),Fréquence des pas par comptage (Hz),Amplitude de l'accélération (g),Ecart-type de la durée des pas (s)
0,Felix,accelerometer7_data.csv,9.99,17,1.901902,1.884570,1.333344,0.078185
1,Felix,accelerometer6_data.csv,9.97,16,1.003009,1.867995,1.193064,0.083484
2,Felix,accelerometer10_data.csv,10.00,14,1.900000,1.589242,1.177756,0.230523
3,Felix,accelerometer1_data.csv,9.99,12,1.001001,1.443570,1.232103,0.213827
4,Felix,accelerometer4_data.csv,10.00,17,1.900000,1.916168,1.248469,0.086658


In [3]:
def make_scatter(df, x, y):
    fig = px.scatter(df, x=x, y=y, color="Nom", hover_data={"Fichier source": True})
    fig.update_traces(textposition='top center', marker=dict(size=15))
    fig.update_layout(
        title=f"{x} vs {y}",
        xaxis_title=x,
        yaxis_title=y,
        hovermode="closest",
    )
    return fig

In [4]:
make_scatter(df, "Fréquence des pas par comptage (Hz)", "Amplitude de l'accélération (g)")

In [5]:
make_scatter(df, "Ecart-type de la durée des pas (s)", "Amplitude de l'accélération (g)")